In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls

adc.json  drive  sample_data


In [0]:
cd drive/Houston/

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
area_hsi = []
for i in range(3):
    for j in range(14):
        img_path = "area/pc_area_" + str(i) + "_" + str(j) + ".png"
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        area_hsi.append(img)
        
area_hsi = np.array(area_hsi)

In [0]:
area_lidar = []
for i in range(14):
    img_path = "area/lidar_area_" + str(i) + ".png"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    area_lidar.append(img)

area_lidar = np.array(area_lidar)

In [0]:
print(area_hsi.shape)
print(area_lidar.shape)

(42, 349, 1905)
(14, 349, 1905)


In [0]:
area_fusion = np.concatenate([area_hsi, area_lidar], axis=0)
print(area_fusion.shape)

(56, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
area_fusion_train_data = []
area_fusion_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            area_fusion_train_data.append(area_fusion[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            area_fusion_test_data.append(area_fusion[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
area_fusion_train_data = np.array(area_fusion_train_data)
area_fusion_test_data = np.array(area_fusion_test_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [0]:
print(area_fusion_train_data.shape)
print(area_fusion_test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 56)
(12197, 56)
(2832,)
(12197,)


In [0]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [0]:
FUSION_PATCH_SIZE = 35
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005
BATCH_SIZE = 25

In [0]:
import numpy as np
padded_area_fusion = np.lib.pad(area_fusion, ((0,0), (FUSION_PATCH_SIZE//2, FUSION_PATCH_SIZE//2), (FUSION_PATCH_SIZE//2,FUSION_PATCH_SIZE//2)), 'reflect')

In [0]:
print(padded_area_fusion.shape)

(56, 383, 1939)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
area_fusion_train_patches = []
area_fusion_test_patches = []


for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      area_fusion_train_patches.append(get_patches(padded_area_fusion, FUSION_PATCH_SIZE, i+FUSION_PATCH_SIZE//2, j+FUSION_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      area_fusion_test_patches.append(get_patches(padded_area_fusion, FUSION_PATCH_SIZE, i+FUSION_PATCH_SIZE//2, j+FUSION_PATCH_SIZE//2))

In [0]:
area_fusion_train_patches = np.array(area_fusion_train_patches)
area_fusion_test_patches = np.array(area_fusion_test_patches)

In [0]:
print(area_fusion_train_patches.shape)
print(area_fusion_test_patches.shape)

(2832, 35, 35, 56)
(12197, 35, 35, 56)


In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = area_fusion_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]


area_fusion_model = Sequential()
area_fusion_model.add(InputLayer(input_shape=(FUSION_PATCH_SIZE, FUSION_PATCH_SIZE, BANDS)))
area_fusion_model.add(Conv2D(kernel_size=3, strides=1, filters=CONV1, padding='same', activation='relu', name='conv1'))
area_fusion_model.add(BatchNormalization())
area_fusion_model.add(MaxPooling2D(pool_size=2, strides=2))
area_fusion_model.add(Conv2D(kernel_size=3, strides=1, filters=CONV2, padding='same', activation='relu', name='conv2'))
area_fusion_model.add(BatchNormalization())
area_fusion_model.add(MaxPooling2D(pool_size=2, strides=2))


area_fusion_model.add(Flatten())
area_fusion_model.add(Dense(FC1, activation='relu'))
area_fusion_model.add(Dropout(0.3))
area_fusion_model.add(Dense(FC2, activation='relu'))
area_fusion_model.add(Dropout(0.1))
area_fusion_model.add(Dense(NUM_CLS, activation='softmax'))

sgd = SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
area_fusion_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [0]:
history = area_fusion_model.fit(area_fusion_train_patches, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=300)

Epoch 1/300
2832/2832 [==============================] - 7s 3ms/step - loss: 2.7906 - acc: 0.0883
Epoch 2/300
2832/2832 [==============================] - 6s 2ms/step - loss: 2.2257 - acc: 0.2087
Epoch 3/300
2832/2832 [==============================] - 6s 2ms/step - loss: 2.0181 - acc: 0.2867
Epoch 4/300
2832/2832 [==============================] - 6s 2ms/step - loss: 1.8982 - acc: 0.3093
Epoch 5/300
2832/2832 [==============================] - 6s 2ms/step - loss: 1.8549 - acc: 0.3234
Epoch 6/300
2832/2832 [==============================] - 6s 2ms/step - loss: 1.7817 - acc: 0.3552
Epoch 7/300
2832/2832 [==============================] - 6s 2ms/step - loss: 1.6267 - acc: 0.4068
Epoch 8/300
2832/2832 [==============================] - 6s 2ms/step - loss: 1.5962 - acc: 0.4220
Epoch 9/300
2832/2832 [==============================] - 6s 2ms/step - loss: 1.5277 - acc: 0.4251
Epoch 10/300
2832/2832 [==============================] - 6s 2ms/step - loss: 1.4628 - acc: 0.4661
Epoch 11/300
2832/2

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [0]:
test_cls = test_labels - 1
prediction = area_fusion_model.predict(area_fusion_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.5727637943756662
Average Accuracy of training samples :  0.5841641605520826
Kappa statistics of training samples :  0.535671822919884
Each class accuracy of training samples :  [0.76349614 0.49022801 0.77822581 0.52483599 0.76445211 0.65882353
 0.68615984 0.46666667 0.4628866  0.6097561  0.46919692 0.56190476
 0.38071066 0.75304878 0.39207048]
Confusion matrix : [[297  51   0  40   0   1   0   0   0   0   0   0   0   0   0]
 [398 602   0 187   0   0  38   0   2   1   0   0   0   0   0]
 [  0   0 386   0  26   0   0   0   0   0   0   0   4   0  80]
 [195 178   0 560   0  29  75  26   1   0   3   0   0   0   0]
 [ 23   0   0  11 886   0   1  40  62   0   0  41   4   0  91]
 [ 45   0   0   0   0 112   1  10   0   0   0   0   0   0   2]
 [  5 101   0  58   0   1 704  15 103   0   8   1   6   0  24]
 [  1  18  17  22  41   0   0 350  53  22   1  57  26   0 142]
 [  7  24   0  35  12   0 169  24 449 106  74  49  16   0   5]
 [  0   3   0   7  19   0 

In [0]:
area_fusion_model.save_weights('Models/area_fusion_model1_weights.h5')